<div style="float: right;">
    <img src="http://www.bigdatacube.org/img/portfolio/rasdaman-logo.png" width="250" style="display: inline-block;"/>
</div>

# BigDataCube Tutorial

This notebook gives a quick overview of the capabilities of the *BigDataCube* service running at CODE-DE. This service is in fact a *rasdaman* server (http://rasdaman.org) which handles requests conforming to one of the following Open Geospatial Consortium (OGC) standards:

- *WCS* (Web Coverage Service)
- *WCPS* (Web Coverage Processing Service)
- *WMS* (Web Map Service)

Requests can address any of the rasdaman datacubes built from Copernicus data at CODE-DE:

- **Sentinel 2**
    - *Date range*: 2018/01/01 - 2019/04/30
    - *CRS*: varying for each datacube depending on the UTM zone it covers
    - *Datacube names*: `S2G5_${utmCrsCode}_${resolution}_${level}`
        - `${utmCrsCode}` - the EPSG code for datacube CRS: 32601 - 32660 (N) and 32701 - 32760 (S)
        - `${resolution}` - 10m, 20m, 60m, or TCI
        - `${level}` - L1C or L2A
        - *Example*: `S2G5_32632_10m_L2A`
- **Sentinel 1 GRD over Germany**
    - *Date range*: 2019/01/01 - 2019/03/24
    - *CRS*: EPSG:4326
    - *Datacube names*: `S1G5_GRD_${modebeam}_${polarisation}`
        - `${modebeam}` - EW, IW
        - `${polarisation}` - HH, HV, VV, VH
        - *Example*: `S1G5_32632_10m_L2A`
- **Sentinel 1 GRD over the North Sea** (sea state and windspeed products produced with DLR's SAINT tool)
    - *Date range*: 2018/01/01 - 2019/03/27
    - *CRS*: EPSG:4326
    - *Datacube names*: `S1_hs_w` (sea state), `S1_windspeed_w` (wind speed)

All OGC requests should be sent to the following endpoint:

- https://processing.code-de.org/rasdaman/ows

The notebook continues with various examples of requests / queries on these datacubes.

## OGC WCS Requests

OGC *WCS* (Web Coverage Service) is a standarized data access protocol on datacubes (*coverages* in OGC terminology). A WCS request can be sent to the BigDataCube server via the base URL below by appending either one of the core operations or some extension like scaling, reprojection, range subsetting, processing, etc:

https://processing.code-de.org/rasdaman/ows?service=WCS&version=2.0.1&request=...

WCS supports 3 core requests: *GetCapabilities* and *DescribeCoverage* help discover available datacubes, while  *GetCoverage* is used to download whole or a part of a coverage, in a chosen resolution / CRS / format encoding.

* **GetCapabilities** - returns XML-encoded description of service properties and a summary of all coverages available on the server:
    
  https://processing.code-de.org/rasdaman/ows?service=WCS&version=2.0.1&request=GetCapabilities
    

* **Describe Coverage** - returns XML-encoded description of a specific coverage, e.g:

  https://processing.code-de.org/rasdaman/ows?service=WCS&version=2.0.1&request=DescribeCoverage&coverageId=S2G5_32632_TCI_L1C


* **GetCoverage** - returns a *full* coverage or a spatio-temporal *subset* (slicing and trimming on coverage axes). This example selects the area between coordinates E(410827.774058,423495.854389) and N(5955107.75272,5963032.77898) on date 2018-05-25, and encodes the result as a PNG image:
    
  https://processing.code-de.org/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID=S2G5_32632_TCI_L1C&SUBSET=ansi(%222018-05-25%22)&FORMAT=image/png&SUBSET=E(410827.774058,423495.854389)&SUBSET=N(5955107.75272,5963032.77898)
    
  <img src="http://www.rasdaman.com/assets/img/cube-slicing.png" width="350" height="250"/>
  <p style="text-align:center;"><b>Trimming / Slicing on 3D data cube to get a subset coverage</b></p>


### How to integrate WCS requests into a python script

You can install required Python libraries for demos with the *pip* tool https://pypi.org/project/pip/.

In [ ]:
# Install required libraries
## *note: osgeolive 13 preinstalled
#pip install requests
#pip install owslib
#pip install numpy


* The *requests* library can be used to send requests to server and parse result (*if in XML*) manually.

In [ ]:
# Using requests library to send GET/POST requests to server
import requests

# Set base url which can be used in further code examples
service_endpoint = "https://processing.code-de.org/rasdaman/ows"
base_wcs_url = service_endpoint + "?service=WCS&version=2.0.1"

result = requests.get(base_wcs_url + "&request=GetCapabilities")

# Result is an XML document which can be parsed to extract 
# information such as coverage identifier, dimensions, axis labels, ...
print(result.text)

* Alternatively, the *OWSlib* library can be used as a more convenient way to parse WCS responses:

In [ ]:
# Get result of GetCapabilities request
from owslib.wcs import WebCoverageService

my_wcs = WebCoverageService(service_endpoint, version="2.0.1")
                            
# Print all available coverages                    
print(my_wcs.contents.keys())

In [ ]:
# Select a coverage (3D irregular coverage) to describe and set it to a reference variable
cov = my_wcs.contents["S2G5_32632_TCI_L1C"]

# Get number of dimensions of coverage
print cov.grid.dimension

In [ ]:
# Get coverage axis labels with order according to coverage's Coordinate Reference System (CRS)
print cov.grid.axislabels

In [ ]:
# Get coverage's time axis's domain (as this coverage is 3D with time axis)
print cov._getTimeLimits()

In [ ]:
# Get coverage's grid lower/upper limits
print cov.grid.lowlimits, cov.grid.highlimits

In [ ]:
# Get offset vectors of coverage (axes' resolutions are non-zero values, respectively)
print cov.grid.offsetvectors

In [ ]:
# Get the list of coefficients for irregular axis (time is an irregular axis of this coverage)
print cov.timepositions

In [ ]:
# Get a subset coverage by slicing on time axis, trimming on spatial axes, and encoding result in image/png.

from IPython.display import Image

request = "&REQUEST=GetCoverage"
cov_id = "&COVERAGEID=S2G5_32632_TCI_L1C"
subset_time = "&SUBSET=ansi(\"2018-05-25\")"
subset_e = "&SUBSET=E(410827.774058,418495.854389)"
subset_n = "&SUBSET=N(5957107.75272,5960032.77898)"
encode_format = "&FORMAT=image/png"

response = requests.get(base_wcs_url + request + cov_id + subset_time + subset_e + subset_n + encode_format)
print base_wcs_url + request + cov_id + subset_time + subset_e + subset_n + encode_format

# Display result directly
Image(data=response.content)

## OGC WCPS Requests

OGC *WCPS* (Web Coverage Processing Service) is an extension of WCS which allows extraction, analysis and filtering of coverages. It establishes a protocol to send a query string (similar to XQuery) to a server and obtain, as a result of the server's processing, a set of result coverages or aggregated scalar values.

The so-called "processing expression" is applied on each of the coverages specified in the given list (*coverageList*), given that the optional filter expression returns true when evaluated on the coverage. Each coverage is referred to in the query by its alias (*variableName*). A processing expression branches down to a miscellanea of possible sub-expressions that are able to return either *scalars (scalar expressions)* or *encoded marrays (coverage expressions)* and operate on both the *data and metadata* of a coverage. More details at http://tutorial.rasdaman.org/rasdaman-and-ogc-ws-tutorial/#ogc-web-services-web-coverage-processing-service.

A WCPS request can be sent to the BigDataCube server via the base URL below by appending a query string:

https://processing.code-de.org/rasdaman/ows?service=WCS&version=2.0.1&request=ProcessCoverages&query=...

The following query is equivalent to the previous WCS request which selects a 2D subset from a 3D coverage, and encodes the result as a PNG image:

```
for $c in (S2G5_32632_TCI_L1C)
return 
  encode(
      $c[ E( 410827.774058 : 423495.854389 ), 
          N( 5955107.75272 : 5963032.77898 ), 
          ansi( "2018-05-25" ) ], 
      "image/png" )
```
*NOTE:* for GET requests, the WCPS query cannot contain new lines or special URL characters like whitespace; the query above is formatted with new lines to make it clearer. To format it correctly before appending to the base URL, the query must be encoded, e.g. with https://meyerweb.com/eric/tools/dencoder/ After encoding, we have the following GET request:

<https://processing.code-de.org/rasdaman/ows?service=WCS&version=2.0.1&request=ProcessCoverages&query=for%20%24c%20in%20(S2G5_32632_TCI_L1C)%0Areturn%20%0A%20%20encode(%0A%20%20%20%20%20%20%24c%5B%20E(%20410827.774058%20%3A%20423495.854389%20)%2C%20%0A%20%20%20%20%20%20%20%20%20%20N(%205955107.75272%20%3A%205963032.77898%20)%2C%20%0A%20%20%20%20%20%20%20%20%20%20ansi(%20%222018-05-25%22%20)%20%5D%2C%20%0A%20%20%20%20%20%20%22image%2Fpng%22%20)>

Step-by-step explanation of the query components:

- `$c` - the coverage variable, an alias for the coverages in the coverage list (only `S2G5_32632_TCI_L1C` in the query above)
- `return` - contains the processing expression, which returns an encoded coverage (as in the example), a scalar value from an aggregation expression, or a metadata value
- `encode` - a function which takes a coverage expression and encodes it to the desired format (PNG in the example)
- `$c[ ... ]` - this is a coverage subsetting expression, where upper and lower limits are specified to restrict the axes in the result as needed; on the time axis a single slice point is specified (2018-05-25) which leads to removing this axis in the result, while on the spatial axes (E and N) we specify upper and lower bounds.

### Change visualization

*WCPS* allows expressing much more complex processing than *WCS*. For example, below we do time-series processing: calculate the difference between B1 bands of 2 datetime slices from the 60m coverage. As can be noticed in the query, we can add, multiply, or do other math operations directly on coverages in order to adjust the calculation; in particularly we do this below in order to fit the result values in the range of [0, 255] for encoding to JPEG.

In [ ]:
query = '''
for $c in (S2G5_32632_60m_L1C) 
return
  encode(
    ( 0.20 * ( 1050.0 + ( 
                (float) $c.B1[ ansi( "2018-05-25" ) ] - 
                        $c.B1[ ansi( "2018-05-10" ) ] ) 
            ) 
    )[ E( 410827.774058 : 423495.854389 ), N( 5955107.75272 : 5963032.77898 ) ]
   , "image/jpeg")
'''

# Send a WCPS query for evaluation on the rasdaman server; as WCPS queries usually contain 
# special characters like '[', ']', '{', '}', it is necessary to send POST requests. 
response = requests.post(service_endpoint, data = {'query': query})

# Display result directly
#print response.content
Image(data=response.content)

### Band math

How about calculating an *NDVI* (Normalized Difference Vegetation Index)? Let's first change the subsetted area to a part of Bremen, Germany, covering both urban and agricultural areas.

In [ ]:
query = '''
for $c in (S2G5_32632_TCI_L1C)
return 
  encode(
      $c[ ansi( "2018-05-25" ), E( 484000 : 493000 ), N( 5874000 : 5877000 ) ] * 1.3, 
      "image/jpeg" )
'''
response = requests.post(service_endpoint, data = {'query': query})
Image(data=response.content)

This NDVI can be derived from the B8 (near-infrared) and B4 (red) bands of the 10m coverage. In the query below we highlight the index values greater than 0.5 (higher vegetation) as white, while everything else is black.

In [ ]:
query = '''
for $c in (S2G5_32632_10m_L1C) 
return
  encode(
    (((float) $c.B8 - $c.B4) / ((float) $c.B8 + $c.B4))
    [ ansi( "2018-05-25" ), E( 484000 : 493000 ), N( 5874000 : 5877000 ) ]
    > 0.5
   , "image/jpeg")
'''
response = requests.post(service_endpoint, data = {'query': query})
Image(data=response.content)

Instead of a black and white boolean map, we can transform the result to a grayscale map where vegetation can be visualized over a greater spectrum (lighter = thicker vegetation). The NDVI values are originaly in the `(-1, 1)` interval, to shift them into a `[0, 255]` interval we can add 1 to the NDVI result and then multiply it by 127.5.

In [ ]:
query = '''
for $c in (S2G5_32632_10m_L1C) 
return
  encode(
    (((((float) $c.B8 - $c.B4) / ((float) $c.B8 + $c.B4)) + 1) * 127.5)
    [ ansi( "2018-05-25" ), E( 484000 : 493000 ), N( 5874000 : 5877000 ) ]
    
   , "image/jpeg")
'''
response = requests.post(service_endpoint, data = {'query': query})
Image(data=response.content)

A *false-color composite* could easily be created from corresponding bands by just specifying them in the desired order:

In [ ]:
query = '''
for $c in (S2G5_32632_10m_L1C) 
return
  encode(
    scale(
      {
         red: $c.B8[ ansi( "2018-05-25" ), E( 484000 : 493000 ), N( 5874000 : 5878000 ) ] / 17.0;
         green: $c.B4[ ansi( "2018-05-25" ), E( 484000 : 493000 ), N( 5874000 : 5878000 ) ] / 17.0;
         blue: $c.B3[ ansi( "2018-05-25" ), E( 484000 : 493000 ), N( 5874000 : 5878000 ) ] / 17.0
      },
      { E:"CRS:1"(0:700) }
    ) 
  , "image/jpeg")
'''
response = requests.post(service_endpoint, data = {'query': query})
Image(data=response.content)

What if the bands we want to combine come from coverages with different resolutions? We can scale the bands to a common resolution before the operations, e.g. below we combine B12 from a 20m coverage, and B8 / B3 from a higher resolution 10m coverage.

In [ ]:
query = '''
for c in (S2G5_32632_20m_L1C), d in (S2G5_32632_10m_L1C)
return
  encode(
    {
      red: scale( c.B12[ ansi( "2018-05-25" ), E( 484000 : 493000 ), N( 5874000 : 5878000 )],
                  { E:"CRS:1"(0:899), N:"CRS:1"(0:399) }) ;
    green: scale( d.B8[ ansi( "2018-05-25" ), E( 484000 : 493000 ), N( 5874000 : 5878000 )],
                  { E:"CRS:1"(0:899), N:"CRS:1"(0:399) }) ;
     blue: scale( d.B3[ ansi( "2018-05-25" ), E( 484000 : 493000 ), N( 5874000 : 5878000 )], 
                  { E:"CRS:1"(0:899), N:"CRS:1"(0:399) })
    } / 15.0
  , "image/jpeg")
'''
response = requests.post(service_endpoint, data = {'query': query})
Image(data=response.content)

### Time-series aggregation

Datacube aggregation is straightforward: select the desired subset and apply an aggregation function such as min, max, avg, count, etc. For example, to calculate the average NDVI value (same spatio-temporal subset as before) the following query can be utilized:

In [ ]:
query = '''
for $c in (S2G5_32632_10m_L1C) 
return
  avg(
    (((float) $c.B8 - $c.B4) / ((float) $c.B8 + $c.B4))
    [ ansi( "2018-05-25" ), E( 484000 : 493000 ), N( 5874000 : 5880000 ) ]
  )
'''
response = requests.post(service_endpoint, data = {'query': query})
print("Average NDVI value: " + response.content)

How about calculating this average for each time point between two dates? This can be done with a *coverage constructor*, which will iterate over all dates between the two given dates, resulting in a 1D array of average NDVI values; notice that the slicing on the time axis `ansi` is done with the "iterator" variable `$pt`. The 1D array is encoded as JSON in the end.

In [ ]:
query = '''
for $c in (S2G5_32632_10m_L1C) 
return
  encode(
    coverage averageNdviValues
    over $pt t(imageCrsDomain($c[ansi("2018-05-20":"2018-05-25")], ansi)) 
    values
      avg(
        (((float) $c.B8 - $c.B4) / ((float) $c.B8 + $c.B4))
        [ ansi( $pt ), E( 484000 : 493000 ), N( 5874000 : 5880000 ) ]
      ),
    "json")
'''
response = requests.post(service_endpoint, data = {'query': query})
print("Average NDVI values: " + response.content)

Furthermore, we could create an aggregated map of several time-slices. The query below selects the maximum NDVI value from each pixel of several time-slices.

In [ ]:
query = '''
for $c in (S2G5_32632_10m_L1C) 
return
  encode(
    (
    condense max
    over $pt t(imageCrsDomain($c[ansi("2018-05-20":"2018-05-25")], ansi)) 
    using
        (((float) $c.B8 - $c.B4) / ((float) $c.B8 + $c.B4))
        [ ansi( $pt ), E( 484000 : 493000 ), N( 5874000 : 5877000 ) ]
    ) > 0.4,
    
    "image/jpeg")
'''
response = requests.post(service_endpoint, data = {'query': query})
Image(data=response.content)

A statistical analysis at a wind park construction site can show the historical ratio of safe working days to support construction planning. For this purpose a datacube from Sentinel 1 GRD data is queried. First we get the average wave height on days where data is available between 2018-01-01 and 2019-03-02 around the existing Amrumbank West wind park near Helgoland in the North Sea.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as pp

query = '''
for $c in (S1_hs_w) return
encode(
   coverage waveHeight
   over $i ansi( imageCrsDomain($c[ansi("2018-01-01" : "2019-03-02")], ansi) )
   values avg($c[ Lat(54.3 : 54.6), Long(7.55 : 7.85), ansi:"CRS:1"($i)])
, "csv")
'''

response = requests.post(service_endpoint, data = {'query': query})

array = np.array(eval(response.text))
pp.plot(array[np.where(array > 0.0)])
pp.show()

The following WCPS query divides the number of time-slices where waves around the wind park area are on average less or equal than 20 decimetres (2 metres) with the total number of time slices available to get a historical ratio of safe working days.

In [ ]:
query = '''
for $c in (S1_hs_w) return
 count(
   coverage safeWaveHeight
   over $i ansi( imageCrsDomain($c[ansi("2018-01-01" : "2019-03-02")], ansi) )
   values (avg($c[ Lat(54.3 : 54.6), Long(7.55 : 7.85), ansi:"CRS:1"($i)]) < 20) and
          (max($c[ Lat(54.3 : 54.6), Long(7.55 : 7.85), ansi:"CRS:1"($i)]) > 0)
 )
 /
 (
   condense +
   over $j ansi( imageCrsDomain($c[ansi("2018-01-01" : "2019-03-02")], ansi) )
   where max($c[ Lat(54.3 : 54.6), Long(7.55 : 7.85), ansi:"CRS:1"($j)]) > 0
   using 1.0
 )
 * 100.0
'''

response = requests.post(service_endpoint, data = {'query': query})
print(response.content + " % of all days are safe for work.")

Wind speed is a main parameter for economic sustainability of a wind park. However, wind wakes from existing parks may reduce wind speed for several dozen kilometres. Using data derived from Sentinel-1 acquisitions, a temporal history of wind speed on a planned construction site can be retrieved, including wake effects which are often not included in wind models.

On the same area as in the previous example, we now go into the `S1_windspeed_w` datacube to calculate average wind speed for each time slice with the following WCPS query:

In [ ]:
query = '''
for $c in (S1_windspeed_w)
return encode(
  coverage avgWindSpeed
  over $i ansi( imageCrsDomain($c[ansi("2018-01-01" : "2019-03-02")], ansi) )
  values avg( $c[ Lat(54.3 : 54.6), Long(7.55 : 7.85), ansi:"CRS:1"($i)] )
, "json")
'''
response = requests.post(service_endpoint, data = {'query': query})
array = np.array(eval(response.text))
pp.plot(array[np.where(array > 0.0)])
pp.show()

We can calculate the correlation between wind speed and sea height with the Pearson correlation coefficient. To do this, we get both the sea state and wind speed averages, and execute a correlation function from scipy:

In [ ]:
from scipy.stats.stats import pearsonr

query = '''
for $c in (S1_windspeed_w), $d in (S1_hs_w)
return encode(
  coverage avgWindSpeed
  over $i ansi( imageCrsDomain($c[ansi("2018-01-01" : "2019-03-02")], ansi) )
  values avg( $c[ Lat(54.3 : 54.6), Long(7.55 : 7.85), ansi:"CRS:1"($i)] )
, "json")
'''
response = requests.post(service_endpoint, data = {'query': query})
windspeed = np.array(eval(response.text))
ws = windspeed[np.where(windspeed > 0.0)]

query = '''
for $c in (S1_windspeed_w), $d in (S1_hs_w)
return encode(
  coverage avgWaveHeight
  over $j ansi( imageCrsDomain($d[ansi("2018-01-01" : "2019-03-02")], ansi) )
  values avg( $d[ Lat(54.3 : 54.6), Long(7.55 : 7.85), ansi:"CRS:1"($j)] )
, "json")
'''
response = requests.post(service_endpoint, data = {'query': query})
waveheight = np.array(eval(response.text))
wh = waveheight[np.where(waveheight > 0.0)]

print("Pearson correlation coefficient: " + str(pearsonr(ws, wh)))

As we can see from the result, there's a moderate positive correlation (0.37), i.e. more wind = higher sea waves.